.

구현목표 : Local pc에 있는 csv 파일을 bastion을 경유하여 private network에 존재하는 RDS에 업로드를 한다

Local pc --> bastion --> RDS


#### STEP 1) RDS 생성

아래와 같은 사양으로 AWS 콘솔에서 RDS를 생성해준다.

Choose a database creation method : standard create

Engine options : MySQL

Version : 8.0.20

Templates : Production

DB instance identifier : pms-rds-test

Master username : admin

Master password : ********

DB instance identifier : pms-rds-test

DB instance size : db.t3.large(2vCPU, 8GB RAM)

Storage : Gerenal Purpose (SSD), 40GB

Enable storage autoscaling : uncheck

Availability & durability : Do not create a stanby instance

VPC : pms-vpc

Subnet group : pms-rds-subnet-group(ap-northeast-2a 영역 private 서브넷이 포함된 subnet group)

public access : No

VPC security groups : deafult

Availability zone : ap-northeast-2a 

Database port : 3306

Database authentication options : password authentication

initial database name : pmstest

DB parameter group : default.mysql8.0

Backup : uncheck

Encryption : uncheck

Perfomance insights : uncheck

Monitoring : check

Log exports : Error log, General log

Enable auto minor version upgrade : uncheck

Maintenance window : No preference

Enable deletion protection : uncheck

#### STEP 2) Bastion 생성

Amazon Linux 2 AMI (HVM), SSD Volume Type

t3.small

vpc : pms-vpc

subnet : pms-public-subnet-a

Auto-assign Public IP : Enable

Name : pms-basion-test

Volume : ssd 8GB

SG : default

key : minmankey

#### STEP 3) RDS 및 bastion 보안그룹 설정

RDS의 보안그룹 inbound rule에서 3306포트로 bastion의 private 아이피를 추가

Bastion의 보안그룹 inbound rule에서 22,3306,3307 포트로 local pc의 아이피를 추가

#### STEP 4) Local port forwarding to RDS

로컬피시에서 terminal을 열고 다음과 같은 명령어를 실행

`ssh -N -L 3307:pms-rds-test.xxxxxxxxxxx.ap-northeast-2.rds.amazonaws.com:3306 ec2-user@ec2-3-35-25-156.ap-northeast-2.compute.amazonaws.com -i minmankey.pem`

--> 별다른 메세지 없이 커서만 껌벅껌벅 거리면 정상적으로 연결이 된것임

#### STEP 5) DBeaver 툴을 이용하여 RDS에 접속

DBeaver 접속 --> connect to database --> MySQL 선택 후 다음 --> port : 3307, server host : localhost, Username : admin, password : * * * * * 입력 후 Test connection 클릭해서 정상 연결여부 확인 --> 완료 클릭 후 접속

#### STEP 6) RDS 생성시 만든 pmstest 디비를 선택하고 다음과 같은 쿼리를 실행한다. 

아래와 같은 쿼리를 실행하여 Table과 메타데이터를 생성해준다.

In [ ]:
CREATE TABLE `minmantable` (
`timestamp_for_kibana` char(30) DEFAULT NULL,
`timestamp` char(30) DEFAULT NULL,
`cpu_usage_percent` float(20) DEFAULT NULL,
`cpu_ctx_switches` float(20) DEFAULT NULL, 
`cpu_interrupts` float(20) DEFAULT NULL, 
`cpu_soft_interrupts` float(20) DEFAULT NULL, 
`cpu_syscalls` float(20) DEFAULT NULL, 
`mem_total` float(20) DEFAULT NULL, 
`mem_available` float(20) DEFAULT NULL, 
`mem_percent` float(20) DEFAULT NULL, 
`mem_used` float(20) DEFAULT NULL, 
`mem_free` float(20) DEFAULT NULL) 
ENGINE=InnoDB DEFAULT CHARSET=utf8;

#### STEP 7) DBeaver 툴을 이용하여 로컬 피시에 있는 csv 파일을 임포트하여 RDS로 넣어주는 작업 

minmantable 우측클릭 --> 데이터 가져오기 클릭 --> CSV 선택후 다음 클릭 --> Source에서 <none> 클릭해서 임포트하고자 하는 csv 파일 선택, Import settings에서 헤더 위치 메뉴에서 헤더를 none으로 할지 top으로 할지 csv 파일에 따라 선택  --> confirm까지 다음 계속 클릭(See how data will be imported into the table 단계에서 mem_free의 경우 데이터가 null이어서 데이터가 비어있기 때문에 Mapping type을 skip 처리를 해줘야함) -->  start 클릭 --> 결과확인

`select * from pmstest.minmantable;` 및 `select count(*) from pmstest.minmantable;` 명령어를 실행하여 정상적으로 csv 데이터가 들어갔는지 확인해본다.